In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import torch

In [2]:
X, y = load_digits(return_X_y=True)

In [3]:
y = np.eye(10)[y]

In [4]:
X = StandardScaler().fit_transform(X)


In [5]:
w1 = torch.repeat_interleave(torch.linspace(0, 1, 10)[None, ...], 64, dim=0)
b1 = torch.zeros(10)
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

In [6]:
for i in range(5):
    
    w1.requires_grad = True
#     b1.requires_grad = True
    z = X_t @ w1 + b1
        
    a = z - torch.max(z, axis=-1, keepdims=True)[0]
    
    y__t = torch.exp(a) / torch.sum(torch.exp(a), keepdims=True, axis=-1)
    assert y__t.shape == y_t.shape
    logy = torch.log(y__t)
    ce = - y_t * logy
    l = torch.sum(ce) / len(y_t)
    l.backward()
#     print(w2.grad)
#     print(b2.grad)
#     print(w1.grad)
#     print(b1.grad)
    if i % 1 == 0:
        print(l)
        
    print(torch.norm(w1.grad))
#     print(a1[0])
    w1 = w1.data - 0.1 * w1.grad
#     w2 = w2.data - 0.01 * w2.grad
    
#     b1 = b1.data - 0.01 * b1.grad
#     b2 = b2.data - 0.01 * b2.grad


    
    

tensor(4.3443, grad_fn=<DivBackward0>)
tensor(1.4740)
tensor(4.1305, grad_fn=<DivBackward0>)
tensor(1.4262)
tensor(3.9305, grad_fn=<DivBackward0>)
tensor(1.3789)
tensor(3.7436, grad_fn=<DivBackward0>)
tensor(1.3322)
tensor(3.5691, grad_fn=<DivBackward0>)
tensor(1.2863)


In [7]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.zeros([in_dim, out_dim])
        self.w = np.repeat(np.linspace(0, 1, 10)[np.newaxis, ...], 64, axis=0)

#         self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([1, out_dim])
        self.dw = None
        self.db = None
        self.in_dim = self.w.shape[0]
        self.out_dim = self.w.shape[1]

        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        self.db = np.mean(d, axis=0)
        assert self.db.shape == self.b.shape, (d.shape, self.db.shape, self.b.shape)
        
        J = np.zeros([self.x.shape[0], self.out_dim, np.prod(self.w.shape)])
        j = 0
        for i in range(self.out_dim):
            J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
            j += self.in_dim
        
        dw = d @ J
        
        dw = np.reshape(np.mean(dw, axis=0), self.w.shape, order='F')
        
        self.dw = dw
        
        d = d @ np.repeat(self.w.T[np.newaxis, ...], d.shape[0], axis=0)
        
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [8]:
class Softmax:
    def __init__(self):
        self.a = None
        
    def forward(self, x):
        assert len(x.shape) == 2
        x = x - np.max(x, axis=-1, keepdims=True)
        self.a = np.exp(x) / np.sum(np.exp(x), keepdims=True, axis=-1)
        return self.a
    def backward(self, d):
        
        diag = np.stack([np.diag(self.a[i]) for i in range(len(self.a))])
        op = np.stack([np.outer(self.a[i], self.a[i]) for i in range(len(self.a))])
        J = diag - op
        
        return d[:, np.newaxis, ...] @ J

In [9]:
class CrossEntropy:
    def forward(self, y_, y):
        
        l = - np.sum(y * np.log(y_))
        l /= len(y)
        return y_, l
    
    def backward(self, y_, y):
        assert y_.shape == y.shape
        d =  - y / y_
        return d

In [10]:
class LogisticRegression:
    def __init__(self):
        self.linear = Linear(64, 10)
        self.softmax = Softmax()
        self.loss = CrossEntropy()
        
    def forward(self, x, y):
        x = self.linear.forward(x)
        x = self.softmax.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self, x, y_, y):
        d = self.loss.backward(y_, y)
        d = self.softmax.backward(d)
        d = self.linear.backward(d)
    
    def step(self, lr):
        self.linear.step(lr)
        print(np.linalg.norm(self.linear.dw))

        

In [11]:
# X = np.array([[2, 1], [1, 2]])
# y = np.array([[0, 1], [1, 0]])


In [12]:
logreg = LogisticRegression()

In [13]:
for i in range(5):
    y_, loss = logreg.forward(X, y)
    logreg.backward(X, y_, y)
    logreg.step(0.1)
    if i % 1 == 0:
        print(loss, 'loss')
#         print((np.argmax(y_, axis=-1) == np.argmax(y, axis=-1)).sum() / len(y_))

1.4740451399377872
4.344260809289213 loss
1.4255308298447877
4.1250148465374545 loss
1.377265098908393
3.919998069832835 loss
1.3294816994324454
3.7286710488975316 loss
1.2823344010915054
3.5504418945072724 loss
